In [1]:
# load and standardize data

import brainscore

neural_data = brainscore.get_assembly(name="dicarlo.Majaj2015")
neural_data.load()
neural_data = neural_data.sel(variation=6).multi_groupby(['category_name', 'object_name', 'image_id']) \
    .mean(dim='presentation').squeeze('time_bin').T
# Mostly, we compare neural data with computational models, 
# for deep neural networks see https://github.com/mschrimpf/brain-score-models.
# This repository is agnostic of the comparison system, 
# To show-case the functionality, we are going to compare different regions.
v4_data = neural_data.sel(region='V4')
it_data = neural_data.sel(region='IT')

# We can compare a set of assemblies directly by calling the metric 
# but for more sophisticated comparisons we will usually build a benchmark.

### Neural fit

In [7]:
# To compare two systems, we instantiate the metric and call it on the source and target assembly.
# The neural fit also relies on training data to instantiate the regression.
from brainscore.metrics.neural_fit import NeuralFit

neural_fit = NeuralFit()
# For demonstration purposes, we will use the same data for training and testing. 
# (which you should obviously never do in practice)
score = neural_fit(v4_data, it_data, v4_data, it_data)
# This gives us a score, containing the correlations per neuroid. 
# For instance, there is one value per cross-validation split.
print("per neuroid: ", score[:10], "...\n")
# Usually we want to aggregate over neuroids to yield a single scalar value:
aggregate = neural_fit.aggregate(score)
print("median over neuroids:", aggregate)


per neuroid:  <xarray.DataAssembly (neuroid_id: 10)>
array([0.431195, 0.5617  , 0.554427, 0.554573, 0.551613, 0.446308, 0.500028,
       0.562305, 0.551316, 0.472134])
Coordinates:
  * neuroid_id  (neuroid_id) object 'Chabo_L_M_5_9' 'Chabo_L_M_6_9' ... ...

median over neuroids: <xarray.DataAssembly ()>
array(0.55406)


### RDM

In [3]:
# We can easily swap out the specific metric and use e.g. RDMs.
# To compare two systems, we instantiate the metric and call it on the source and target assembly.
from brainscore.metrics.rdm import RDMMetric

rdm = RDMMetric()
score = rdm(v4_data, it_data)
print(score)
# Note how the score is much lower with RDMs due to missing re-mapping.

<xarray.DataAssembly ()>
array(0.289892)


### Benchmarks

In [8]:
# So far, we have compared data directly and gotten a single score.
# But we are not sure how reliable this score is. 
# And we might want to compare with V4 and IT data jointly without having to split them manually.
# This is what Benchmarks are for: they automatically compare multiple streams of data and cross-validate.
# We can access pre-defined benchmarks by their name:

from brainscore import benchmarks

benchmark = benchmarks.load('dicarlo.Majaj2015')
# And then just pass in the assembly to compare:
score = benchmark(it_data)
# This gives us a score, containing the raw `values` that went into the score. 
# For instance, there is one value per cross-validation split.
print("raw score values: ", score.values, "\n")
# We often care only about a value and error bars, 
# so the score also contains an `aggregation` with a center and error (typically mean and s.e.m.).
# Since the score is potentially multi-dimensional, it is also structured as an xarray DataArray.
# We can select the center as follows:
print("center:", score.aggregation.sel(aggregation='center'))

Stratification coord 'None' not found in assembly - falling back to un-stratified splits


Stratification coord 'None' not found in assembly - falling back to un-stratified splits


raw score values:  <xarray.DataAssembly (region: 2, split: 10, neuroid_id: 256)>
array([[[1.020304, 0.953583, ...,      nan,      nan],
        [1.143224, 0.99301 , ...,      nan,      nan],
        ...,
        [1.065376, 0.924344, ...,      nan,      nan],
        [1.000426, 0.966133, ...,      nan,      nan]],

       [[     nan,      nan, ..., 0.867279, 0.885616],
        [     nan,      nan, ..., 0.875509, 0.900431],
        ...,
        [     nan,      nan, ..., 0.845986, 0.878324],
        [     nan,      nan, ..., 0.856884, 0.994246]]])
Coordinates:
  * neuroid_id  (neuroid_id) object 'Chabo_L_A_2_4' 'Chabo_L_A_3_3' ...
  * region      (region) object 'IT' 'V4'
  * split       (split) int64 0 1 2 3 4 5 6 7 8 9 

center: <xarray.DataAssembly (region: 2)>
array([1.011786, 0.686611])
Coordinates:
    aggregation  <U6 'center'
  * region       (region) object 'IT' 'V4'


In [10]:
# We can also define our own benchmarks.
# If you just want to put together existing assemblies and metrics in new ways, use the `build` method:
my_benchmark = benchmarks.build(assembly_name='dicarlo.Majaj2015', metric_name='rdm', target_splits=['subregion'])
# If you want to use new data, a new metric etc., you should call the benchmark constructor.
# Here, we want to keep the transformations (i.e. cross-validation), so we'll use SplitBenchmark:

from brainscore import benchmarks
from brainscore.benchmarks import SplitBenchmark
from brainscore.metrics.neural_fit import NeuralFit
from brainscore.metrics.ceiling import ceilings

assembly_loader = benchmarks._assemblies['dicarlo.Majaj2015']
assembly = assembly_loader(average_repetition=False).sel(region='IT')
metric = NeuralFit()
ceiling = ceilings['splitrep'](metric, average_repetition=assembly_loader.average_repetition)
my_benchmark = SplitBenchmark(target_assembly=assembly, metric=metric, ceiling=ceiling, target_splits=['region'])